In [1]:
import re

import matplotlib.pyplot as plt
import pandas as pd

## Очистка данных

### Данные пациентов (1 табл)

In [2]:
patients = pd.read_csv("../data/raw/данные_пациентов.csv")

print("=" * 80)
print("HEAD".center(80))
print("=" * 80)
print(patients.head())

print("\n" + "=" * 80)
print("INFO".center(80))
print("=" * 80)
print(patients.info())

                                      HEAD                                      
   id_пациента дата_рождения пол район_проживания           регион
0            2    08.05.1958   Ж       ПУШКИНСКИЙ  Санкт-Петербург
1            8    28.10.1957   М       ВЫБОРГСКИЙ  Санкт-Петербург
2            9    03.05.1953   Ж      КАЛИНИНСКИЙ  Санкт-Петербург
3           10    30.05.1981   М       ПРИМОРСКИЙ  Санкт-Петербург
4           12    02.03.1980   М      КАЛИНИНСКИЙ  Санкт-Петербург

                                      INFO                                      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413799 entries, 0 to 413798
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id_пациента       413799 non-null  int64 
 1   дата_рождения     413799 non-null  object
 2   пол               413799 non-null  object
 3   район_проживания  413799 non-null  object
 4   регион            413799 non-null  obj

In [3]:
patients_clean = patients.drop_duplicates(keep="first")
patients_clean.shape

(379246, 5)

Пациенты могли переезжать. Предполагаем, что данные отранжированы по времени, и оставляем только последние записи для каждого пациента в каждом районе проживания.

In [4]:
patients_clean = patients_clean.drop_duplicates(subset=["id_пациента"], keep="last")
patients_clean.shape

(379076, 5)

In [5]:
patients_clean.columns = ["id_patient", "birth_date", "gender", "residential_area", "region"]

In [7]:
patients_clean.to_csv("../data/processed/patients.csv", index=False)

### Данные рецептов

In [18]:
recipes = pd.read_csv("../data/raw/данные_рецептов.csv")

print("=" * 80)
print("HEAD".center(80))
print("=" * 80)
print(recipes.head())

print("\n" + "=" * 80)
print("INFO".center(80))
print("=" * 80)
print(recipes.info())

                                      HEAD                                      
   id_пациента         дата_рецепта код_диагноза  код_препарата  id_пациента.1
0     24419779  2022-03-31 11:52:30          F06      200900018        1226228
1     17890856  2019-06-04 18:41:10        I11.9      201300167          62857
2     18749669  2019-10-29 12:35:31        K86.1      201300027        1186134
3     31412579  2024-07-30 17:06:59        I67.8      202200024        1927759
4     21191943  2020-12-01 19:59:02        E10.9      201200267        1265025

                                      INFO                                      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   id_пациента    1000000 non-null  int64 
 1   дата_рецепта   998822 non-null   object
 2   код_диагноза   1000000 non-null  object
 3   код_препарата  1000000

`id_пациента.1` не нужен

In [19]:
recipes = recipes.drop(columns=["id_пациента"])
recipes = recipes.rename(columns={"id_пациента.1": "id_пациента"})

In [20]:
recipes_id = set(recipes["id_пациента"])
patients_id = set(patients["id_пациента"])

intersection_ids = recipes_id.intersection(patients_id)

print(f"Уникальные id в рецептах: {len(recipes_id)}")
print(f"Уникальные id в пациентах: {len(patients_id)}")
print(f"Количество общих id: {len(intersection_ids)}")
print(f"Процент общих id: {len(intersection_ids) / len(patients_id) * 100:.2f}%")

Уникальные id в рецептах: 114228
Уникальные id в пациентах: 379076
Количество общих id: 114228
Процент общих id: 30.13%


ранжируем по дате выписки рецепта

In [21]:
recipes["дата_рецепта"] = pd.to_datetime(recipes["дата_рецепта"], errors="coerce")
recipes_sorted = recipes.sort_values("дата_рецепта")

удаляем дубликаты

In [22]:
cleaned_recipes = recipes_sorted.drop_duplicates(keep="first")

In [23]:
cleaned_recipes.columns = ["date", "diagnosis_code", "medication_code", "id_patient"]

In [24]:
cleaned_recipes = cleaned_recipes.sort_values(["id_patient", "date"])

# Для каждого пациента заполняем NaN последней известной датой
cleaned_recipes["date"] = cleaned_recipes.groupby("id_patient")["date"].ffill()

# Удаляем записи, где date все еще NaN (нет ни одной даты для пациента)
cleaned_recipes = cleaned_recipes.dropna(subset=["date"])

print(f"Осталось записей: {len(cleaned_recipes)}")
print(f"NaN значений в date: {cleaned_recipes['date'].isna().sum()}")

cleaned_recipes.isna().sum()

Осталось записей: 915851
NaN значений в date: 0


date               0
diagnosis_code     0
medication_code    0
id_patient         0
dtype: int64

In [27]:
cleaned_recipes["diagnosis_code"] = cleaned_recipes["diagnosis_code"].str.upper()

In [29]:
cleaned_recipes.to_csv("../data/processed/recipes.csv", index=False)

### Данные препаратов

In [30]:
medication = pd.read_csv("../data/raw/данные_препараты.csv")

print("=" * 80)
print("HEAD".center(80))
print("=" * 80)
print(medication.head())

print("\n" + "=" * 80)
print("INFO".center(80))
print("=" * 80)
print(medication.info())

                                      HEAD                                      
   код_препарата дозировка Торговое название  стоимость  \
0      200052200       1.5       Галоперидол      12.98   
1      200052400         5       Галоперидол      15.51   
2      200059000       3.5           Манинил      93.42   
3      200165100        50        Пури-Нетол     525.48   
4      200194100      0.01         Нифедипин      12.60   

                    Полное_название  
0          Галоперидол, 1,5 мг № 50  
1  Галоперидол, 5 мг № 50, таблетки  
2   Манинил, 3,5 мг № 120, таблетки  
3  Пури-Нетол, 50 мг № 25, таблетки  
4   Нифедипин, 10 мг № 50, таблетки  

                                      INFO                                      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3393 entries, 0 to 3392
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   код_препарата      3393 non-null   int64  
 1  

In [31]:
medication.duplicated(subset=["код_препарата"]).sum()

0

In [32]:
medication.columns = ["medication_code", "dosage", "trade_name", "price", "info"]

In [38]:
medication["dosage"] = medication["dosage"].fillna("Unknown")

In [47]:
medication["price"] = medication["price"].fillna(0)

In [49]:
medication.to_csv("../data/processed/medication.csv", index=False)

### Данные диагнозов

In [50]:
diagnoses = pd.read_csv("../data/raw/данные_диагнозы.csv")

print("=" * 80)
print("HEAD".center(80))
print("=" * 80)
print(diagnoses.head())

print("\n" + "=" * 80)
print("INFO".center(80))
print("=" * 80)
print(diagnoses.info())

                                      HEAD                                      
  код_мкб                                  название_диагноза  \
0     A00                                             Холера   
1   A00.0  Холера, вызванная холерным вибрионом 01, биова...   
2   A00.1  Холера, вызванная холерным вибрионом 01, биова...   
3   A00.9                                Холера неуточненная   
4     A01                                      Тиф и паратиф   

                               класс_заболевания  
0  НЕКОТОРЫЕ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ  
1  НЕКОТОРЫЕ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ  
2  НЕКОТОРЫЕ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ  
3  НЕКОТОРЫЕ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ  
4  НЕКОТОРЫЕ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ  

                                      INFO                                      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14801 entries, 0 to 14800
Data columns (total 3 columns):
 #   Column             Non-Null Count

In [51]:
diagnoses.columns = ["diagnosis_code", "name", "classification"]

In [53]:
diagnoses["diagnosis_code"] = diagnoses["diagnosis_code"].str.upper()

In [55]:
diagnoses.to_csv("../data/processed/diagnoses.csv", index=False)

## Инсайты